<a href="https://colab.research.google.com/github/yunjinchoidev/ai/blob/main/%ED%85%90%EC%84%9C%ED%94%8C%EB%A1%9C_%EC%82%AC%EC%A7%84_%EB%B6%84%EB%A5%98%EA%B8%B0_%EB%A7%8C%EB%93%A4%EA%B8%B0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf

In [2]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()

11490434/11490434 [==============================] - 0s 0us/step


In [3]:
x_train, x_test = x_train.astype('float32'), x_test.astype('float32')

In [4]:
x_train, x_test = x_train.reshape([-1, 784]), x_test.reshape([-1, 784])

In [6]:
x_train, x_test = x_train / 255. , x_test/255.

In [7]:
y_train, y_test = tf.one_hot(y_train, depth=10), tf.one_hot(y_test, depth=10)

In [8]:
train_data = tf.data.Dataset.from_tensor_slices((x_train, y_train))

In [9]:
train_data = train_data.repeat().shuffle(60000).batch(100)

In [13]:
train_data_iter = iter(train_data)

In [34]:
# 모델 만들기

class SoftmaxRegression(tf.keras.Model):
  def __init__(self):
    super(SoftmaxRegression, self).__init__()
    self.softmax_layer = tf.keras.layers.Dense(10,
                                               activation=None,
                                               kernel_initializer='zeros',
                                               bias_initializer='zeros')

  def call(self, x):
    logits = self.softmax_layer(x)

    return tf.nn.softmax(logits)

In [36]:
## 손실함수

@tf.function
def cross_entropy_loss(y_pred, y):
  return tf.reduce_mean(-tf.reduce_sum(y*tf.math.log(y_pred), axis=[1]))

In [37]:
# 옵티마이저
optimizer = tf.optimizers.SGD(0.5)

In [38]:
# 훈련 함수
@tf.function
def train_step(model, x, y):
  with tf.GradientTape() as tape:
    y_pred = model(x) # 예측값
    loss = cross_entropy_loss(y_pred, y) # 오차 계산히기
  gradient = tape.gradient(loss, model.trainable_variables) # 그래디언트 계산하기
  optimizer.apply_gradients(zip(gradient, model.trainable_variables)) # 옵티마이저 적용

In [39]:
# 정확도 출력 함수

@tf.function
def compute_accuracy(y_pred, y):
  correct_prediction = tf.equal(tf.argmax(y_pred,1), tf.argmax(y,1))
  accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

  return accuracy

In [40]:
# 모델 선언 
SoftmaxRegression_model = SoftmaxRegression()

In [42]:
for i in range(1000):
  batch_xs, batch_ys = next(train_data_iter)
  train_step(SoftmaxRegression_model, batch_xs, batch_ys) # 훈련함수에 모델과 데이터를 넣어서 돌린다.

In [44]:
print("정확도(Accuracy): %f" % compute_accuracy(SoftmaxRegression_model(x_test), y_test)) # 정확도 : 약 91%

정확도(Accuracy): 0.919900
